- 참조

  - 텐서플로우 공식 홈페이지  
    https://www.tensorflow.org/tutorials/generative/cyclegan


# CycleGAN

- 조건부 GAN을 사용하여 페어링되지 않은 이미지 대 이미지 변환

- 한 이미지 도메인의 특성을 캡처하고 이러한 특성을 다른 이미지 도메인으로 변환 할 수있는 방법을 알아낼 수있는 방법을 제안

- Pix2Pix와의 차이는 추가 손실 함수와 페어링되지 않은 훈련 데이터의 사용한다는 점

- CycleGAN은 a cycle consistency loss를 사용하여 쌍을 이루는 데이터 없이도 훈련을 가능하게 함  
  즉, 소스 및 대상 도메인 간의 일대일 매핑없이 한 도메인에서 다른 도메인으로 변환 가능

- 이것은 사진 향상, 이미지 색상 화, 스타일 전송 등과 같은 많은 흥미로운 작업을 수행 할 수있는 가능성을 열어줌  
  필요한 것은 소스와 대상 데이터 세트 (단순히 이미지 디렉토리)뿐임

  <img src="https://www.tensorflow.org/tutorials/generative/images/horse2zebra_1.png">

  <img src="https://www.tensorflow.org/tutorials/generative/images/horse2zebra_2.png">

- Pix2Pix의 generator와 discriminator를 가져올 수 있는 패키지 설치
* https://github.com/tensorflow/examples.git

## 입력 파이프 라인 

- 학습 데이터셋에 random jittering을 적용

  - 이는 과적합을 방지하는 몇 가지 이미지 확대 기술

  - Pix2Pix와 유사

    - 임의 jittering에서 286x286 크기의 이미지가 무작위로 256x256으로 잘림

    - 랜덤 미러링에서는 이미지가 좌우로 무작위로 반전됨

- 이미지 값을 [-1, 1] 사이의 값으로 정규화

## Pix2Pix 모델 로드 및 재사용

- Pix2Pix에서 사용된 generator와 discriminator를 가져옴

- Pix2Pix와의 차이점

  - CycleGAN은 배치 정규화 대신 인스턴스 정규화 사용

  - 논문에서는 ResNet기반 생성기를 사용하지만 본 코드에서는 UNet 생성기를 기반으로 함

- 2 개의 생성기 (G 및 F)와 2 개의 판별 기 (X 및 Y)가 여기에서 훈

  - 생성기 `G` 는 이미지 `X`를 이미지 `Y`로 변환하는 방법을 학습 $(G : X -> Y)$

  - 생성기 `F`는 이미지 `Y`를 이미지 `X`로 변환하는 방법을 학습 $(F : Y -> X)$

  - 판별기 `D_X`는 이미지 `X`와 생성 된 이미지 `X`( `F(Y)` )를 구별하는 방법을 학습

  -  판별기 `D_Y` 는 이미지 `Y`와 생성 된 이미지 `Y`( `G(X)` )를 구별하는 `D_Y` 학습

  <img src="https://www.tensorflow.org/tutorials/generative/images/cyclegan_model.png">

## 손실 기능

- CycleGAN에서는 학습 할 쌍데이터가 없으므로 입력 `x` 및 대상 `y` 쌍이 학습 중에 의미가 있다는 보장이 없음  

- 따라서 네트워크가 올바른 매핑을 학습하도록하기 위해 논문 저자는 주기 일관성 손실을 제안

- 판별기 손실과 생성기 손실은 pix2pix 에서 사용되는 것과 유사

### 주기 일관성

- 주기 일관성은 결과가 원래 입력에 가까워 야 함을 의미

- 예를 들어, 한 문장을 영어에서 프랑스어로 번역 한 다음 다시 프랑스어에서 영어로 번역하는 경우 결과 문장은 원래 문장과 동일해야 함

- 주기 일관성 손실에서

  - $X$이미지는 생성 된 $hatY$이미지를 생성하는 $G$생성기를 통해 전달

  - 생성 된 이미지 $hatY$는 순환 이미지 $hatX$을 생성하는 생성기 $F$를 통해 전달

  - 평균 절대 오차는 $X$와 $hatX$사이에서 계산됨
  
  $\quad forward\ cycle\ consistency\ loss: X -> G(X) -> F(G(X)) \sim \hat{X}$

  $\quad backward\ cycle\ consistency\ loss: Y -> F(Y) -> G(F(Y)) \sim \hat{Y}$

  <img src="https://www.tensorflow.org/tutorials/generative/images/cycle_loss.png">

- 위와 같이 $G$ 생성기는 $X$ 이미지를 $Y$ 이미지로 변환

- ID 손실은 $Y$ 이미지를 생성기 $G$ 에 공급하면 실제 이미지 $Y$ 또는 $Y$ 에 가까운 이미지를 생성해야 한다고 말함

- 말에서 zebra-to-horse 모델을 실행하거나 얼룩말에서 horse-to-zebra 모델을 실행하는 경우 이미지에 이미 대상 클래스가 포함되어 있으므로 이미지를 많이 수정해서는 안됨


$$Identity\ loss = |G(Y) - Y| + |F(X) - X|$$

- 모든 생성기와 판별기에 대한 옵티마이저 초기화

## 체크 포인트 설정

## 학습

- 논문에서는 `Epochs=200`으로 설정

- 여기서는 `40`으로 설정. 예측이 덜 정확할 수 있음

- 학습 과정 
  
  - 예측을 가져옴

  - 손실을 계산

  - 역 전파를 사용하여 기울기를 계산

  - 옵티마이저에 그라디언트를 적용

- 학습 소요시간 매우 오래 걸림

## 테스트 데이터셋을 사용하여 생성